# Autoencoders - Colorization

This notebook shows autoencoders for colorization of black and white images. The flowers data is used to ilustrate the concept.

The images are initially converted to Lab color space. The input is the L channel, and the target are the (a,b) channels.

Transfer learning is used for the encoder. A VGG16 with imagenet weights is used as the base model for the encoder.

- dataset: https://www.tensorflow.org/datasets/catalog/tf_flowers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import numpy as np

import os
import pathlib
import matplotlib.pyplot as plt

# to display confusion matrix
import seaborn as sn
import pandas as pd

import cv2

## Auxiliary functions 

In [ ]:
def show_sample():
    columns = 5
    rows = 5
    plt.figure(figsize=(20, 20))
    n = 1
    for d in os.listdir(data):
        fs = os.listdir(f'{data}{d}')
        for f in fs[:5]:
            img = cv2.imread(f'{data}{d}/{f}')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
            ax = plt.subplot(rows, columns, n)
            plt.imshow((img))
            plt.axis('off')
            n += 1
        
        
def showLAB(filename):
    
    img = cv2.imread(filename)
    img = cv2.resize(img, (WIDTH,HEIGHT))
    LAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    plt.figure(figsize=(15, 60))
    ax = plt.subplot(1,4,1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("RGB");
    ax = plt.subplot(1,4,2)
    plt.title("L");
    plt.imshow(LAB[:,:,0], cmap='gray')
    ax = plt.subplot(1,4,3)
    plt.title("a");
    plt.imshow(LAB[:,:,1], cmap='gray')
    ax = plt.subplot(1,4,4)
    plt.title("b");
    plt.imshow(LAB[:,:,2], cmap='gray')
   
        
def eval_images(filenames):
    
    k = len(filenames)
    rows = k
    cols = 3
    print(rows,cols)
    f, axes= plt.subplots(rows,cols,figsize=(4 * cols,4 * rows))
    fig=plt.figure()
    for a in range (len(filenames)):
        img = cv2.imread(filenames[a])
        img = cv2.resize(img, (WIDTH,HEIGHT))
        LAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

        x = LAB[:,:,0] 
        x = x.reshape((WIDTH, HEIGHT, 1))
        x = np.concatenate((x,)*3, axis=-1)
        xx = tf.keras.applications.vgg16.preprocess_input(x)

        ab = model.predict(xx.reshape(1,WIDTH,HEIGHT,3))

        cur = np.zeros((192, 192, 3))
        cur[:,:,0] = x[:,:,0] / 255.0
        cur[:,:,1:] = ab 
        cur = np.clip((cur * 255).astype(np.ubyte), 0, 255)

        rgb = cv2.cvtColor(cur, cv2.COLOR_Lab2RGB)
        
        subplot_title="original"
        axes.ravel()[3*a].set_title(subplot_title)  
        axes.ravel()[3*a].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        subplot_title="bw input"
        axes.ravel()[3*a+1].set_title(subplot_title)  
        axes.ravel()[3*a+1].imshow(x)
        subplot_title="output"
        axes.ravel()[3*a+2].set_title(subplot_title)  
        axes.ravel()[3*a+2].imshow(rgb)
    fig.tight_layout()    
    plt.show()
    
    

In [ ]:
def prepare_callbacks(file_path):

    checkpointer = ModelCheckpoint(filepath= file_path, 
                               monitor = 'val_loss',
                               verbose=1, 
                               save_best_only = True)


    earlyStopper = EarlyStopping(monitor='val_loss',  
        min_delta = 0.00001, patience = 20, verbose = 1)

    reduceLR = ReduceLROnPlateau(monitor='val_loss', 
         factor=0.5, patience=5, min_lr=0.0000001, verbose = 1)

    return [checkpointer, earlyStopper, reduceLR]
    

## Configuration

In [ ]:
HEIGHT = 192
WIDTH = 192
NUM_CHANNELS = 3
BATCH_SIZE = 32

In [ ]:
data_test = 'D:/deepLearning/datasets/flowers/test_ARF/'
data = 'D:/deepLearning/datasets/flowers/train/'
model_prefix = 'd:/vcpi/DL07/flowers/'

train_online = False